In [7]:
from delta_rest_client import DeltaRestClient, OrderType
import json
import sys
import requests
from time import time
import hmac
import hashlib
from utils import *
import pytz

key = None
secret = None
with open("creds.json",'r') as f:
    op = json.load(f)
    key = op.get('api')
    secret = op.get('secret')

if key is None or secret is None:
    print(key)
    print(secret)
    sys.exit(1)
baseurl = "https://api.india.delta.exchange"

In [8]:
client = DeltaRestClient(
  base_url=baseurl,
  api_key=key,
  api_secret=secret
)
productId = get_product_id_by_symbol(symbol = 'BTCUSD')
client.set_leverage(productId, 20)

KeyboardInterrupt: 

In [ ]:
class symbol_management:
    def __init__(self, client, productId):
        self.productId = productId
        self.client = client

    def recordorder(self, entry, stoploss, qty, target, tradetime, tradetype, openorderid):
        data = None
        with open("trades.json", "r") as f:
            data = json.load(f)
        if data.get(tradetime) is not None:
            return 
        data['trade']={
            "tradetime" : tradetime,
            "entry": entry,
            "stoploss": stoploss,
            "target": target,
            "tradetype": tradetype,
            "qty" : qty,
            "openorderid" : openorderid,
            "stoplossorderid" : None,
            "targetorderid" : None,
            "istradeactive" : True
        }
        with open("trades.json", "w") as f:
            json.dump(data, f, indent=2)
            print("saved data")
        
    
    def trade(self, entry, stoploss, tradetype, target, tradetime, qty):
   
        if not self._is_any_active_trade():
            print('No active position')
            trades = None
            with open("trades.json", "r") as f:
                trades = json.load(f)
            
            if trades.get('trade') is None:
                order = self._place_order(tradetype, entry, qty)
                self.recordorder(entry, stoploss, qty, target, tradetime, tradetype, order['id'])

            else:
                self._cancel_pending_orders(trades['trade']['stoplossorderid'], trades['trade']['targetorderid'])
                if self._is_order_open(trades['trade']['openorderid']):
                    self._cancel_order(trades['trade']['openorderid'])
                    print('Cancelling previous open order')
                    with open("trades.json", "w") as f:
                        json.dump({}, f, indent=2)
                
                order = self._place_order(tradetype, entry, qty)
                self.recordorder(entry, stoploss, qty, target, tradetime, tradetype, order['id'])

                
    def _state_manage(self):
        if not self._is_any_active_trade():
            return
        trades = None
        with open("trades.json", "r") as f:
            trades = json.load(f)
        
        latest_trade = trades.get('trade')
        if latest_trade is None:
            print("There is no saved trade")
            return 
        
        for i in self.client.order_history(page_size=10)['result']:
            if i['id'] == latest_trade['openorderid'] and latest_trade['istradeactive'] == True:
                print('Position id found : ', i)
                if i['state'] == 'cancelled':
                    latest_trade['istradeactive'] == False
            
                if i['state'] == 'closed':
                    print("Position executed Now create sl and taret order")
                    if self._is_any_active_trade():
                        if latest_trade['stoplossorderid'] is None:
                            latest_trade['stoplossorderid'] = self.validate_stoploss_order(latest_trade['stoploss'], latest_trade['tradetype'], latest_trade['qty'])
                        if latest_trade['targetorderid'] is None:
                            latest_trade['targetorderid'] = self.validate_target_order(latest_trade['target'], latest_trade['tradetype'], latest_trade['qty'])
                
                trades['trade'] = latest_trade
                with open("trades.json", "w") as f:
                    json.dump(trades, f, indent=2)
                      
    def _is_any_active_trade(self):
        r = self.client.get_position(self.productId)
        if r['size'] == 0:
            return False
        return True 
    
    def _is_order_open(self, id):
        r =  client.get_live_orders()
        for orders in r:
            if  orders['order_type']=='limit_order' and orders['state'] == 'open' and orders['product_id'] == self.productId and orders['id'] == id:
                return True
        return False
    
    def _cancel_all_open_orders(self):
        r =  client.get_live_orders()
        for orders in r:
            if  orders['order_type']=='limit_order' and orders['state'] == 'open' and orders['product_id'] == self.productId:
                self._cancel_order(orders['id'])

    def _cancel_order(self, orderid):
        self.client.cancel_order(productId, orderid)
    
    def print_position_state(self):
        print("product id : ", self.productId)
        print("isactiveposition : ", self.isactiveposition)
        print("stoplossid : ", self.stoplossid)
        print("targetid : ",self.targetid)
        print("opentrade : ", self.opentradeid)
        print("myposition state : ", self.activeTradeState) 
    
    def _cancel_pending_orders(self, stoplossorderid, targetorderid):
        r =  client.get_live_orders()
        for orders in r:
            if orders['stop_order_type'] == 'take_profit_order' and orders['state'] == 'pending' and orders['product_id'] == self.productId and stoplossorderid is not None and stoplossorderid == orders['id']:
                self._cancel_order(stoplossorderid)
            elif orders['stop_order_type'] == 'stop_loss_order' and orders['state'] == 'pending' and orders['product_id'] == self.productId and targetorderid is not None and targetorderid == orders['id']:
                self._cancel_order(targetorderid)

    # def _load_open_orders(self):
    #     r =  client.get_live_orders()
    #     for orders in r:
    #         if  orders['order_type']=='limit_order' and orders['state'] == 'open' and orders['product_id'] == self.productId:
    #             self.opentradeid= orders['id']  

    def _place_order(self, tradetype, entry, qty):
        order = self.client.place_order(self.productId, qty, tradetype, limit_price=entry, time_in_force=None, order_type=OrderType.LIMIT)
        print("Limit order placed : " , order)
        return order
    
    def validate_stoploss_order(self, stoploss,tradetype, qty):
        stop_order = self.client.place_stop_order(
        product_id=self.productId,
        size=qty,
        side='sell' if tradetype == 'buy' else 'buy',
        stop_price=stoploss,
        limit_price=stoploss,
        order_type=OrderType.MARKET,
        isTrailingStopLoss=False
        )
        return stop_order['id']
        
    def validate_target_order(self, target,tradetype, qty):    
        target_order = self.client.place_order(
        product_id=self.productId,
        size=qty,
        side='sell' if tradetype == 'buy' else 'buy',
        limit_price=target,
        order_type=OrderType.LIMIT,
        )

        return target_order['id']

In [9]:
import requests
import time
from datetime import datetime, timedelta
import pandas as pd
import pytz
symbolmanager = symbol_management(client, productId)

class Strategy:
    def __init__(self, symbol, rr = 1.5, timer = 3):
        self.symbol = symbol
        self.rr = rr
        self.timer = timer

    def _get_unix_timestamp(self, dt):
        return int(time.mktime(dt.timetuple()))

    def _load_historical_data(self, interval='3m', days=1):
        end_dt = datetime.now() + timedelta(minutes=1)
        start_dt = end_dt - timedelta(days=days)

        start_unix = self._get_unix_timestamp(start_dt)
        end_unix = self._get_unix_timestamp(end_dt)

        url = 'https://api.india.delta.exchange/v2/history/candles'
        params = {
            'resolution': interval,
            'symbol': self.symbol,
            'start': start_unix,
            'end': end_unix
        }
        headers = {'Accept': 'application/json'}
        response = requests.get(url, params=params, headers=headers)

        if response.status_code == 200:
            data = response.json()
            df = pd.DataFrame(data['result'])
            df['time'] = pd.to_datetime(df['time'], unit='s',  utc=True).dt.tz_convert(pytz.timezone('Asia/Kolkata'))
            df = df[::-1]
            return df
        else:
            raise ConnectionError(f"Failed to fetch data: {response.status_code} - {response.text}")

    def run_strategy(self):
        try:
            df = self._load_historical_data()
            # df.tail(2).head(1)
            print(df.tail(1))
            return 
            df['prevcandle'] = (df['close'] < df['high'].shift(1)) & (df['close'] > df['low'].shift(1))
            # df['buy'] = (df['close'] > df['high'].shift(1)) & (df['prevcandle'].shift(1)) & (df['prevcandle'].shift(1))
            # df['sell'] = (df['close'] < df['low'].shift(1)) & (df['prevcandle'].shift(1)) & (df['prevcandle'].shift(1))
            df['buy'] = (df['close'] > df['high'].shift(1)) 
            df['sell'] = (df['close'] < df['low'].shift(1)) 
            df['time'] = df['time'].dt.strftime('%Y-%m-%d %H:%M:%S')
            
            # print(df.tail(5))
            # return
            # print(df.tail(4))
            df = df.tail(2).head(1)
            print(df)
            row = df.iloc[0]

            if row['buy']:
                entry = row['close'] - 10
                stoploss = row['low'] - 30
                target = entry + self.rr * (entry - stoploss)
                symbolmanager.trade(entry, stoploss, 'buy', target,row['time'][:16], 5)

            if row['sell']:
                entry = row['close'] + 10
                stoploss = row['high'] + 30
                target = entry - self.rr * (stoploss - entry)
                symbolmanager.trade(entry, stoploss, 'sell', target,row['time'][:16], 5)

        except Exception as e:
            print(f"Error running strategy: {e}")

    def start(self):
        while(True):
            if datetime.today().minute % self.timer <= 3:
                self.run_strategy()
                symbolmanager._state_manage()
                time.sleep(5)
            else:
                time.sleep(5)

    def stop(self):
        self.running = False

strategy = Strategy('BTCUSD')
strategy.start()

      close      high       low      open                      time  volume
0  119050.5  119050.5  119003.5  119042.0 2025-07-14 05:27:00+05:30    1004
      close      high       low      open                      time  volume
0  119050.5  119050.5  119003.5  119042.0 2025-07-14 05:27:00+05:30    1004
      close      high       low      open                      time  volume
0  119050.5  119050.5  119003.5  119042.0 2025-07-14 05:27:00+05:30    1004
      close      high       low      open                      time  volume
0  119050.5  119050.5  119003.5  119042.0 2025-07-14 05:27:00+05:30    1004
      close      high       low      open                      time  volume
0  119050.5  119050.5  119003.5  119042.0 2025-07-14 05:27:00+05:30    1004
      close      high       low      open                      time  volume
0  119050.5  119050.5  119003.5  119042.0 2025-07-14 05:27:00+05:30    1004
      close      high       low      open                      time  volume
0  119050.5 

KeyboardInterrupt: 